# *MunchXMLmuncher* Alpha
Developed by research assistant Loke Sjølie for the University of Oslo
___
Users:
I ask that you do not touch anything under the header **Program** unless you *sort of* know what you're doing. :)

Developers:
I apologize in advance.
## Options

In [99]:
# Users: only edit things that exist WITHIN double quotation marks ("").
newtitle = "MunchXMLmuncher Alpha Test Result" # Title of resulting CMIF
editorName = "Loke Sjølie" # The name to issue to the CMIF file as "editor" (responsible for this file)
editorMail = "loke.sjolie@ub.uio.no" # The e-mail associated with the above.

version = "0.1" # Describes the "program's" state of completion and versioning.
cmifUid = "a403c593-09df-4538-8acf-8d459339fca8" # Unique ID. Used in sourceDesc of CMIF. Don't change it without a good reason.

inputfilepath = r"INPUT/tei.xml" # Where is the target TEI source file located?

## Program

### Init, metadata, etc

In [70]:
from bs4 import BeautifulSoup # Hent BeautifulSoup-modulen (https://www.crummy.com/software/BeautifulSoup/) for XML
from bs4 import Comment # BS4-addon for å håndtere kommentarer <!-- X -->
from datetime import date # Dates
#import glob # The yeast of thought and mind
#import os # Filsystem for lasting og lagring
today = date.today() # Sett dato i dag
today = today.strftime("%Y-%m-%d") # Formater dato
currVer = version+" "+today

In [71]:
previouslyRun = "Last executed code was version "+str(version)+" on date "+str(today)+". All OUTPUT files are current to that version on that date.\n"+str(cmifUid)+"."
print("Version",currVer)
try:
    with open(r"settings/previous", "r", encoding="utf-8") as file: # Åpne filen med encoding UTF8
        f = file.readlines() # Les innholdet som linjer
        f = "".join(f) # Linjene blir kombinert i en variabel
    if f == previouslyRun:
        print("> Re-run detected. If you've changed anything, make sure to update the version var.")
    else:
        f = open(r"settings/previous", "w", encoding="utf-8")
        f.write(previouslyRun)
        print("> Updated internal files to reflect the date and version we're running today.")
    file.close()
except:
    f = open(r"settings/previous", "w", encoding="utf-8")
    f.write(previouslyRun)
    f.close()
    print("> I made a new file to keep track of when the script was last run.")

Version 0.0 2022-08-11
> Updated internal files to reflect the date and version we're running today.


### Read/process TEI-XML

In [72]:
with open(inputfilepath, "r", encoding="utf-8") as file: # Open a file
    tei = file.readlines() # Les innholdet som linjer
    tei = "".join(tei) # Linjene blir kombinert i en variabel
soup = BeautifulSoup(tei, from_encoding="UTF-8") # It is now soup

#### Debug stuff - Tags/attributes
What is this thing? An investigation of the input document's tags.

In [78]:
tagsAttrs = [] # New list
for x in soup.findAll(): # For every tag in the soup
    tag = str(x.name) # Assign name of tag to var tag
    for attribute in x.attrs: # For every attribute belonging to the tag
        tag = tag+" @"+attribute # Append attribute to tag with " @" as separator - results in combination
    if tag not in tagsAttrs: # If this particular combination of tag/attribute(s) has not been seen previously
        tagsAttrs.append(tag) # Register it in our list
# Dict with known tag/attribute pairings and understood meanings
dict = {
    "tei":"The TEI element - is where our file actually begins.",
    "teiheader":"The TEI header contains metadata (titleStmt, publicationStmt, sourceDesc...).",
    "p":"P is a paragraph. This is used in the TEIheader to contain the actual strings for publication & source desc. And a single, random </p> element later.",
    "body":"Body is used as a sub-element of <text> to contain all the metadata for all letters. I am personally offended by this practice. BS4 adds one, too.",
    "text":"Text appears to be a wrapper for the body tag, which contains all the texts' metadata.",
    "div":"Div, with @xml:id, is used to contain the metadata of a single letter.",
    "date":"Date is a date element. It seems to have the @when attribute very often, as well as enclosed text. Often has @type(year/fromTo, etc.)",
    "table":"Table is the primary data structure in which information about each letter is stored. This is a *table*.",
    "row":"Row is a sub-element of the table element. It defines a new X-axis in a table.",
    "seg":"Seg appears to be some kind of ID attached to each letter. The ID is used as an @xml:id attribute in div, and the element appears in references to other letters.",
    "cell":"Cell is a sub-element of the table element also. A single cell appears to be an entry into a row element.",
    "ref":"Ref appears to contain references to other XML items.",
    "item":"Item is a generic element that has multiple @attributes, such as owner, owner signature, author, paper type... This is evidently a very important tag.",
    "list":"List is a list. Often, the list only has one item. The list is used as description tag, containing other lists, and describes anything between dates to material type.",
    "html":"The HTML tag can be ignored. BS4 adds this.",
    "filedesc":"Filedesc contains title, publication, source statements.",
    "sourcedesc":"Sourcedesc describes the source of the whole document.",
    "publicationstmt":"Publication statement for the whole document.",
    "title":"Title for the whole document.",
    "titlestmt":"Titlestmt is a wrapper for the title tag (whole document).",
    "div @xml:id":"Div has an attribute @xml:id. This describes the unique ID of the item in question.",
    "list @type":"List's @type attribute describes whether the list is wrapped around an object/physical description, a date, or other category.",
    "item @n":"Item's @n attribute describes role, library sorting, language, measures, dated, notes and so on. Very... multipurpose.",
    "tei @xmlns @xml:id":"tei @xmlns @xml:id is functionally identical to TEI tag. Just the one.",
    "date @type @from @to":"date @type @from @to describes the sequence type=fromTo, from, to. A date range.",
    "ref @target":"ref's @target attribute describes a URL to another XML.",
    "date @type @when":"Date with attributes type and when. Single date/year.",
    "ref @type @target":"Seems to contain URL to eMunch's web pages for a 'Read More' function.",
    "date @type @from":"date @type @from is an open-ended date.",
    "date @type":"Caution: date @type is a date with just a type. The date itself might be enclosed...? Potentially misleading. Investigate.",
    "ref @target @n":"ref @target @n - like ref @target, but @n tends to be the name of an institution or so.",
    "row @n":"row @n describes parts of the text. Inventory number, paper type, etc.",
    "ref @type @target @n":"ref @type @target @n - Working off of previous information, I'll infer that ref @type @target @n describes a Read More, with URL, with name."
}
print("Listing all unique tags and attribute combinations found with mapped, understood meanings.\n")
tagsAttrs.sort() # We do a little sorting
for x in tagsAttrs: # For every tag/attr combination registered
    if x in dict: # If our dict has the combo
        if "@" in x: # If there's an attribute involved
            print("ATTR ["+str(x)+"]",dict[x]) # Print with attribute focus
        else: # If there is no attribute involved
            print("TAG ["+str(x)+"]",dict[x]) # Print with tag focus
    else: # If our dict does not have the combo
        print("\n"+str(x),"has no description. What is this?\n") # Print error
comments = soup.find_all(string=lambda text: isinstance(text, Comment)) # Find all comments in soup
if comments: # If there are comments
    n = len(comments) # Check how many comments
    print("\n> Detected",n,"comments (<!-- -->, etc). These should be eradicated before tag extraction.") # Print message
else: # If there are no comments
    print("\n> There are no (0) comments to worry about in this document.") # There are no comments
print("\n> You have... you've put an item in a list in an item to describe single years... *why*?") # Complain a bit

Listing all unique tags and attribute combinations found with mapped, understood meanings.

TAG [body] Body is used as a sub-element of <text> to contain all the metadata for all letters. I am personally offended by this practice. BS4 adds one, too.
TAG [cell] Cell is a sub-element of the table element also. A single cell appears to be an entry into a row element.
TAG [date] Date is a date element. It seems to have the @when attribute very often, as well as enclosed text. Often has @type(year/fromTo, etc.)
ATTR [date @type] Caution: date @type is a date with just a type. The date itself might be enclosed...? Potentially misleading. Investigate.
ATTR [date @type @from] date @type @from is an open-ended date.
ATTR [date @type @from @to] date @type @from @to describes the sequence type=fromTo, from, to. A date range.
ATTR [date @type @when] Date with attributes type and when. Single date/year.
TAG [div] Div, with @xml:id, is used to contain the metadata of a single letter.
ATTR [div @xml:

We have notes as well as letters. The notes generally do not have a item @recipient tag, while the letters generally do.

Every div has an xml:id, and an enclosed ID.

Every item then has a list with items in it.

#### Debug stuff - Types of attributes
There's a whole lot of item *n* tags. What are they? Let's find out. The following extracts list and item tags with unique attribute texts. We have to filter out a loooot of tags that're IDs, dates etc. And look - we got cells, too!

In [151]:
itemNs = []
for x in soup.findAll(True):
    if ("target" not in str(x.attrs)) and ("xml:id" not in str(x.attrs)) and ("from" not in str(x.attrs)) and ("year" not in str(x.attrs)) and ("day" not in str(x.attrs)) and ("month" not in str(x.attrs)):
        name = x.name
        for i in x.attrs:
            attribute = i
            value = x.attrs[i]
            try:
                contents = x.contents[0]
                fullTag = str(name)+" @"+str(attribute)+" ("+str(value)+") "+str(contents)
            except:
                fullTag = str(name)+" @"+str(attribute)+" ("+str(value)+")"
            if fullTag not in itemNs:
                itemNs.append(fullTag)
itemNs

['list @type (proveniens) \n',
 'item @n (ownerMsSig) MM N 4',
 'item @n (owner) Munchmuseet',
 'list @type (description) \n',
 'item @n (author) Edvard Munch',
 'list @type (objectType) <item n="notes">notat</item>',
 'item @n (notes) notat',
 'list @type (date) <item n="dated">\n<date from="1889" to="1890" type="fromTo">1889–1890</date>\n</item>',
 'item @n (dated) \n',
 'item @n (language) norsk',
 'list @type (physDescription) \n',
 'item @n (paperType) skrivepapir',
 'item @n (measures) 178×111 mm (h×b)',
 'item @n (ownerMsSig) MM N 5',
 'item @n (measures) 173×220 mm (h×b)',
 'list @type (relatedObjects) \n',
 'item @n (ownerMsSig) MM N 6',
 'item @n (ownerMsSig) MM N 7',
 'item @n (ownerMsSig) MM N 8',
 'item @n (measures) 173×112 mm (h×b)',
 'item @n (ownerMsSig) MM N 9',
 'item @n (measures) 173×110 mm (h×b)',
 'item @n (ownerMsSig) MM N 10',
 'item @n (ownerMsSig) MM N 11',
 'item @n (ownerMsSig) MM N 12',
 'item @n (ownerMsSig) MM N 13',
 'item @n (ownerMsSig) MM N 14',
 'it

In [148]:
x.contents[0]

'skrivepapir'

TypeError: 'builtin_function_or_method' object is not subscriptable

### CMIF

https://correspsearch.net/en/documentation.html

/correspAction/@type == correspAction element with attribute type="xyz"

/correspAction/persName == correspAction element with persName child element

@X == attribute of element

*Each letter, postcard - document - that is to be described features its own **correspDesc element**. *There are as many correspDescs as there are items. A particular correspDesc element in CMI format is more restrictive and reduced with regard to its vocabulary than the TEI Guidlines generally allow. This enables interchange between the respective TEI documents.*

for each in letters:
    create correspDesc wrapper
    
<correspDesc>
    <correspAction type="sent">
        <persName ref="VIAFetc url">NAME</>
        <placeName ref="Geonames url">NAME</>
    <correspAction type="received">
        <persName ref="url">NAME</>
        <placeName ref="Geonames url">NAME</>

### Mapping tags
*Italics* == Tag is category/folder only, does not contain text in itself

#### TEI-Header (metadata)
1. *Titlestmt* {Title, Editor(email)}
2. *Publicationstmt* {*Publisher* (Ref @target), idno@url, date@when, *Availability*(licence@target)}
3. *Sourcedesc* {Bibl@type@xml:id} - type="online" xml:id="cmifUid"

The header mostly features direct correlation, or items where the program will directly inject new information.

Now, because nothing is easy, the example file is just all TEI header including the letters it wants to describe. There is a body tag with a random \<p/>, which just serves absolutely no purpose. Why?

#### "profileDesc" (data)
1. correspDesc @key @ref @source {correspAction @type (persname @ref, placename @ref, *date @when*), correspAction @type (persname @ref, placename @ref, *date @when*)}

Dates need to be YYYY-MM-DD, dropping DD and/or MM if required. Unknown dates should be skipped as per CMIF documentation. 
